In [1]:
!pip install gurobipy

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import numpy as np
import gurobipy as gp

# Juice Glass Example

In [3]:
ojMod=gp.Model()
ojModX = ojMod.addMVar(2)
ojMod.setObjective(500*ojModX[0] + 450*ojModX[1],sense=gp.GRB.MAXIMIZE)
conlist=[0]*3
conlist[0] = ojMod.addConstr(6*ojModX[0] + 5*ojModX[1] <= 60)
conlist[1] = ojMod.addConstr(1000*ojModX[0] + 2000*ojModX[1] <= 15000)
conlist[2] = ojMod.addConstr(ojModX[0] <= 8)
ojMod.Params.OutputFlag = 0 
ojMod.optimize() 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19


In [4]:
ojMod.objVal # print the optimal objective value

5142.857142857143

In [5]:
ojModX.x # print the optimal decision variables

array([6.42857143, 4.28571429])

# Shadow Prices

In [6]:
[conlist[0].Pi, conlist[1].Pi, conlist[2].Pi]

[array(78.57142857), array(0.02857143), array(0.)]

In [7]:
[con.Pi for con in conlist]

[array(78.57142857), array(0.02857143), array(0.)]

# Lower and upper bound where shadow prices are valid

In [8]:
[con.SARHSLow for con in conlist]

[array(37.5), array(12800.), array(6.42857143)]

In [9]:
[con.SARHSUp for con in conlist]

[array(65.5), array(24000.), array(inf)]

## Let's see if the shadow price actually works like I told you it would?

In [10]:
db = 7
ojMod.Objval + db*conlist[0].Pi

5692.857142857143

In [11]:
# now try to re-pose and resolve with 67 hours of production capacity
ojMod2=gp.Model()
ojModX2 = ojMod2.addMVar(2)
ojMod2.setObjective(500*ojModX2[0] + 450*ojModX2[1],sense=gp.GRB.MAXIMIZE)
conlist2=[0]*3
conlist2[0] = ojMod2.addConstr(6*ojModX2[0] + 5*ojModX2[1] <= 60+db) # add db to the constraint!
conlist2[1] = ojMod2.addConstr(1000*ojModX2[0] + 2000*ojModX2[1] <= 15000)
conlist2[2] = ojMod2.addConstr(ojModX2[0] <= 8)
ojMod2.Params.OutputFlag = 0 
ojMod2.optimize() 
ojMod2.objVal

5575.0

# Range of obj where optimal corner stays the same

In [12]:
ojModX.SAObjLow

array([225.        , 416.66666667])

In [13]:
ojModX.SAObjUp

array([ 540., 1000.])